In [1]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os, random, json
spark=SparkSession.builder.master("local").appName("studentmarkpractice").getOrCreate()

In [2]:
class landing_to_unification:
    
    """unification"""
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter

    def write_parquet(self,df):
        df.write.format(self.file_parameter['dest_format']).mode("overwrite").option("path",self.file_parameter['dest_path']).saveAsTable("school_unified.{0}".format(self.file_parameter['view_name']))
        
    def main_block(self):
        
        """main execution block"""
        temp_obj=common_ingestion(self.file_parameter)
        df=temp_obj.read_input_file()
        temp_obj.create_view(df)
        

In [3]:
if __name__=='__main__':
    file_parameter1={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/student.parquet",
        "view_name":"studentdetails"
    }
    
    file_parameter2={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/subject.parquet",
        "view_name":"subjectdetails"
    }
    
    file_parameter3={
        "src_format":"parquet",
        "src_path":"C:/Git_Repo/big_data_project/data_files/landing/school_landing/marks.parquet",
        "view_name":"markdetails"
    }
    file_parameter4={
        "dest_format":"parquet",
        "dest_path":"C:/Git_Repo/big_data_project/data_files/unified/school_unified/overallreport.parquet",
        "view_name":"school"
    }
    
    %run C:/Git_Repo/big_data_project/etl_scripts/common_ingestion/common_ingestion.ipynb
    
    spark.sql("create database if not exists {0}".format("school_unified"))
    unified_obj1=landing_to_unification(file_parameter1)
    unified_obj2=landing_to_unification(file_parameter2)
    unified_obj3=landing_to_unification(file_parameter3)
    unified_obj4=landing_to_unification(file_parameter4)
    unified_obj1.main_block()
    unified_obj2.main_block()
    unified_obj3.main_block()
    sql_query=open("C:/Git_Repo/big_data_project/config/landing_sql.sql","r")
    stud_df=spark.sql(sql_query.read())
    unified_obj4.write_parquet(stud_df)
    